In [125]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori, association_rules


Loading the dataset

In [126]:
sales_receipts = pd.read_csv('coffee_data/201904 sales reciepts.csv')
sales_receipts

,transaction_id,transaction_date,transaction_time,sales_outlet_id,staff_id,customer_id,instore_yn,order,line_item_id,product_id,quantity,line_item_amount,unit_price,promo_item_yn
0,7,2019-04-01,12:04:43,3,12,558,N,1,1,52,1,2.50,2.50,N
1,11,2019-04-01,15:54:39,3,17,781,N,1,1,27,2,7.00,3.50,N
2,19,2019-04-01,14:34:59,3,17,788,Y,1,1,46,2,5.00,2.50,N
3,32,2019-04-01,16:06:04,3,12,683,N,1,1,23,2,5.00,2.50,N
4,33,2019-04-01,19:18:37,3,17,99,Y,1,1,34,1,2.45,2.45,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49889,753,2019-04-29,16:51:58,8,42,0,N,1,1,30,1,3.00,3.00,N
49890,756,2019-04-29,16:51:14,8,42,8412,Y,1,1,25,2,4.40,2.20,N
49891,759,2019-04-29,11:17:36,8,15,0,Y,1,1,31,1,2.20,2.20,N
49892,763,2019-04-29,15:45:52,8,45,8030,N,1,1,44,2,5.00,2.50,N


In [127]:
all_products = pd.read_csv('coffee_data/product.csv')
all_products

,product_id,product_group,product_category,product_type,product,product_description,unit_of_measure,current_wholesale_price,current_retail_price,tax_exempt_yn,promo_yn,new_product_yn
0,1,Whole Bean/Teas,Coffee beans,Organic Beans,Brazilian - Organic,It's like Carnival in a cup. Clean and smooth.,12 oz,14.40,$18.00,Y,N,N
1,2,Whole Bean/Teas,Coffee beans,House blend Beans,Our Old Time Diner Blend,Out packed blend of beans that is reminiscent ...,12 oz,14.40,$18.00,Y,N,N
2,3,Whole Bean/Teas,Coffee beans,Espresso Beans,Espresso Roast,Our house blend for a good espresso shot.,1 lb,11.80,$14.75,Y,N,N
3,4,Whole Bean/Teas,Coffee beans,Espresso Beans,Primo Espresso Roast,Our primium single source of hand roasted beans.,1 lb,16.36,$20.45,Y,N,N
4,5,Whole Bean/Teas,Coffee beans,Gourmet Beans,Columbian Medium Roast,A smooth cup of coffee any time of day.,1 lb,12.00,$15.00,Y,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...
83,84,Add-ons,Flavours,Regular syrup,Chocolate syrup,Bursting with chocolate flavour,pump,0.04,$0.80,Y,N,N
84,85,Beverages,Coffee,Specialty coffee,Rio Nights,2 shots of Ouro Brasilerio and pure cane sugar...,6 oz,1.20,$6.00,Y,N,Y
85,87,Beverages,Coffee,Barista Espresso,Ouro Brasileiro shot,From Rio,1.5 oz,0.60,$3.00,Y,N,N
86,88,Beverages,Coffee,Barista Espresso,Ouro Brasileiro shot promo,Ouro promo,16 oz,0.42,$2.10,Y,Y,N


Data Wrangler

In [128]:
sales_receipts = sales_receipts[['transaction_id','transaction_date','sales_outlet_id','customer_id', 'product_id','quantity']]
all_products = all_products[['product_id','product_category','product']]
data = pd.merge(sales_receipts, all_products, on='product_id',how='left')

Filtering Data based on sizes like Rg, Lg, sm

In [129]:
data['product']=data['product'].str.replace(' Rg','')
data['product']=data['product'].str.replace(' Lg','')
data['product']=data['product'].str.replace(' Sm','')

data['product'].unique()

array(['Traditional Blend Chai', 'Brazilian', 'Serenity Green Tea',
       'Our Old Time Diner Blend', 'Jamaican Coffee River', 'Ethiopia',
       'English Breakfast', 'Sustainably Grown Organic', 'Earl Grey',
       'Cappuccino', 'Espresso shot', 'Latte', 'Dark chocolate',
       'Columbian Medium Roast', 'Oatmeal Scone', 'Morning Sunrise Chai',
       'Peppermint', 'Jumbo Savory Scone', 'Lemon Grass',
       'Chocolate Chip Biscotti', 'Spicy Eye Opener Chai',
       'Ginger Biscotti', 'Chocolate Croissant', 'Hazelnut Biscotti',
       'Cranberry Scone', 'Scottish Cream Scone ', 'Croissant',
       'Almond Croissant', 'Ginger Scone', 'Ouro Brasileiro shot',
       'Organic Decaf Blend', 'Chocolate syrup', 'Hazelnut syrup',
       'Carmel syrup', 'Sugar Free Vanilla syrup',
       'Jamacian Coffee River', 'Guatemalan Sustainably Grown',
       'Civet Cat', 'Chili Mayan', 'Primo Espresso Roast',
       'Brazilian - Organic', 'I Need My Bean! Diner mug',
       'Espresso Roast', 'I Need 

Filtering products to keep only coffee options

In [130]:
coffee_products = [
    'Dark chocolate',
    'Cappuccino',               
    'Espresso shot',            
    'Latte',                   
    'Oatmeal Scone',           
    'Jumbo Savory Scone',       
    'Chocolate Chip Biscotti', 
    'Ginger Biscotti',          
    'Chocolate Croissant',      
    'Hazelnut Biscotti',        
    'Cranberry Scone',         
    'Scottish Cream Scone ',   
    'Croissant',                
    'Almond Croissant',        
    'Ginger Scone',           
    'Ouro Brasileiro shot',    
    'Chocolate syrup',
    'Hazelnut syrup',
    'Carmel syrup'
]

data = data[data['product'].isin(coffee_products)]

Transactions handling

In [131]:
data.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot
20,127,2019-04-01,3,116,41,2,Coffee,Cappuccino
21,134,2019-04-01,3,189,38,2,Coffee,Latte
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino


Adding new field transaction to concatenate transaction id and customer id to handle group ordering cases

In [132]:
data['transaction']=data['transaction_id'].astype(str)+'_'+data['customer_id'].astype(str)
data.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product,transaction
16,108,2019-04-01,3,65,40,1,Coffee,Cappuccino,108_65
17,112,2019-04-01,3,90,37,2,Coffee,Espresso shot,112_90
20,127,2019-04-01,3,116,41,2,Coffee,Cappuccino,127_116
21,134,2019-04-01,3,189,38,2,Coffee,Latte,134_189
22,135,2019-04-01,3,131,40,1,Coffee,Cappuccino,135_131


In [133]:
items_bought = data['transaction'].value_counts().reset_index()
valid_items = items_bought[items_bought['count']>1]['transaction'].tolist()
data = data[data['transaction'].isin(valid_items)]
data.head()

,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,product_category,product,transaction
34,199,2019-04-01,3,112,41,2,Coffee,Cappuccino,199_112
35,199,2019-04-01,3,112,79,1,Bakery,Jumbo Savory Scone,199_112
54,296,2019-04-01,3,328,39,1,Coffee,Latte,296_328
55,296,2019-04-01,3,328,79,1,Bakery,Jumbo Savory Scone,296_328
64,357,2019-04-01,3,530,41,2,Coffee,Cappuccino,357_530


Products analysis

In [134]:
data['product_category'].value_counts()

product_category
Bakery                4322
Coffee                3530
Flavours              1641
Drinking Chocolate     970
Packaged Chocolate      23
Name: count, dtype: int64

In [135]:
data['product'].value_counts()

product
Cappuccino                 1225
Latte                      1207
Dark chocolate              993
Chocolate Croissant         643
Espresso shot               593
Chocolate syrup             568
Carmel syrup                561
Ginger Scone                520
Hazelnut syrup              512
Ouro Brasileiro shot        505
Jumbo Savory Scone          366
Chocolate Chip Biscotti     362
Croissant                   360
Cranberry Scone             360
Almond Croissant            355
Scottish Cream Scone        354
Hazelnut Biscotti           342
Oatmeal Scone               341
Ginger Biscotti             319
Name: count, dtype: int64

Recommendation Engine

In [136]:
product_recommendations = data.groupby(['product','product_category']).count().reset_index()
product_recommendations

,product,product_category,transaction_id,transaction_date,sales_outlet_id,customer_id,product_id,quantity,transaction
0,Almond Croissant,Bakery,355,355,355,355,355,355,355
1,Cappuccino,Coffee,1225,1225,1225,1225,1225,1225,1225
2,Carmel syrup,Flavours,561,561,561,561,561,561,561
3,Chocolate Chip Biscotti,Bakery,362,362,362,362,362,362,362
4,Chocolate Croissant,Bakery,643,643,643,643,643,643,643
5,Chocolate syrup,Flavours,568,568,568,568,568,568,568
6,Cranberry Scone,Bakery,360,360,360,360,360,360,360
7,Croissant,Bakery,360,360,360,360,360,360,360
8,Dark chocolate,Drinking Chocolate,970,970,970,970,970,970,970
9,Dark chocolate,Packaged Chocolate,23,23,23,23,23,23,23


In [137]:
product_recommendations = product_recommendations[['product','product_category','transaction_id']]
product_recommendations = product_recommendations.rename(columns={'transaction_id':'number_of_transactions'})
product_recommendations

,product,product_category,number_of_transactions
0,Almond Croissant,Bakery,355
1,Cappuccino,Coffee,1225
2,Carmel syrup,Flavours,561
3,Chocolate Chip Biscotti,Bakery,362
4,Chocolate Croissant,Bakery,643
5,Chocolate syrup,Flavours,568
6,Cranberry Scone,Bakery,360
7,Croissant,Bakery,360
8,Dark chocolate,Drinking Chocolate,970
9,Dark chocolate,Packaged Chocolate,23


Saving to csv file

In [138]:
product_recommendations.to_csv('api/objects/product_recommendations.csv',index=False)

Apriori Engine

In [139]:
train_data = data.groupby(['transaction','product'])['product'].count().reset_index(name='Count')
train_data.head()

,transaction,product,Count
0,1000_0,Dark chocolate,1
1,1000_0,Oatmeal Scone,1
2,1001_8306,Cappuccino,1
3,1001_8306,Carmel syrup,1
4,1002_0,Carmel syrup,1


In [140]:
basket = train_data.pivot_table(index='transaction',columns='product',values='Count').fillna(0)
basket.head()

product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Ouro Brasileiro shot,Scottish Cream Scone
transaction,,,,,,,,,,,,,,,,,,,
1000_0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1001_8306,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1002_0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1003_0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1004_5383,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [141]:
def encoding(val):
    if val>0:
        return 1
    else:
        return 0

In [142]:
basket=basket.map(encoding)
basket.head()


product,Almond Croissant,Cappuccino,Carmel syrup,Chocolate Chip Biscotti,Chocolate Croissant,Chocolate syrup,Cranberry Scone,Croissant,Dark chocolate,Espresso shot,Ginger Biscotti,Ginger Scone,Hazelnut Biscotti,Hazelnut syrup,Jumbo Savory Scone,Latte,Oatmeal Scone,Ouro Brasileiro shot,Scottish Cream Scone
transaction,,,,,,,,,,,,,,,,,,,
1000_0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
1001_8306,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1002_0,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0
1003_0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1004_5383,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0


In [143]:
frequent_itemsets = apriori(basket, min_support=0.05, use_colnames=True)
frequent_itemsets.head().sort_values(by='support',ascending=False)

c:\Users\Chinmay Purohit\AppData\Local\Programs\Python\Python313\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
1,0.357169,(Cappuccino)
2,0.187477,(Carmel syrup)
4,0.135606,(Chocolate Croissant)
0,0.116339,(Almond Croissant)
3,0.114116,(Chocolate Chip Biscotti)


In [144]:
rules_basket = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules_basket.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(Almond Croissant),(Cappuccino),0.116339,0.357169,0.052242,0.449045,1.257232,0.010689,1.166756,0.231539
1,(Cappuccino),(Almond Croissant),0.357169,0.116339,0.052242,0.146266,1.257232,0.010689,1.035053,0.318282
2,(Almond Croissant),(Dark chocolate),0.116339,0.280474,0.056317,0.484076,1.725921,0.023687,1.394636,0.475974
3,(Dark chocolate),(Almond Croissant),0.280474,0.116339,0.056317,0.200793,1.725921,0.023687,1.105671,0.584551
4,(Almond Croissant),(Latte),0.116339,0.356428,0.053353,0.458599,1.286651,0.011886,1.188715,0.252120


In [145]:
rules_basket[rules_basket['antecedents']=={'Cappuccino'}].sort_values(by='lift',ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
12,(Cappuccino),(Chocolate syrup),0.357169,0.185254,0.107447,0.300830,1.623880,0.041280,1.165305,0.597655
6,(Cappuccino),(Carmel syrup),0.357169,0.187477,0.100778,0.282158,1.505027,0.033817,1.131896,0.522004
20,(Cappuccino),(Hazelnut syrup),0.357169,0.168210,0.088551,0.247925,1.473900,0.028472,1.105993,0.500175
18,(Cappuccino),(Ginger Biscotti),0.357169,0.105965,0.052612,0.147303,1.390107,0.014765,1.048479,0.436555
16,(Cappuccino),(Croissant),0.357169,0.113746,0.055206,0.154564,1.358857,0.014579,1.048281,0.410819
9,(Cappuccino),(Chocolate Chip Biscotti),0.357169,0.114116,0.055206,0.154564,1.354445,0.014447,1.047843,0.407090
22,(Cappuccino),(Jumbo Savory Scone),0.357169,0.115969,0.054094,0.151452,1.305974,0.012674,1.041817,0.364463
1,(Cappuccino),(Almond Croissant),0.357169,0.116339,0.052242,0.146266,1.257232,0.010689,1.035053,0.318282
25,(Cappuccino),(Scottish Cream Scone ),0.357169,0.115969,0.051130,0.143154,1.234413,0.009710,1.031726,0.295410
15,(Cappuccino),(Cranberry Scone),0.357169,0.117451,0.051501,0.144191,1.227669,0.009551,1.031245,0.288487


Generate recommended items in a dictonary format

In [146]:
product_categories = data[['product','product_category']].drop_duplicates().set_index('product').to_dict()['product_category']
product_categories

{'Cappuccino': 'Coffee',
 'Jumbo Savory Scone': 'Bakery',
 'Latte': 'Coffee',
 'Chocolate Chip Biscotti': 'Bakery',
 'Espresso shot': 'Coffee',
 'Hazelnut Biscotti': 'Bakery',
 'Chocolate Croissant': 'Bakery',
 'Dark chocolate': 'Packaged Chocolate',
 'Cranberry Scone': 'Bakery',
 'Croissant': 'Bakery',
 'Scottish Cream Scone ': 'Bakery',
 'Almond Croissant': 'Bakery',
 'Ouro Brasileiro shot': 'Coffee',
 'Ginger Biscotti': 'Bakery',
 'Oatmeal Scone': 'Bakery',
 'Ginger Scone': 'Bakery',
 'Chocolate syrup': 'Flavours',
 'Hazelnut syrup': 'Flavours',
 'Carmel syrup': 'Flavours'}

In [147]:
recommendation_dict = {}
antecedents = rules_basket['antecedents'].unique()
for antecedent in antecedents:
    dataFrame = rules_basket[rules_basket['antecedents']==antecedent]
    dataFrame = dataFrame.sort_values(by='confidence',ascending=False)
    key = "_".join(antecedent)
    recommendation_dict[key] = []
    for index, row in dataFrame.iterrows():
        consequents = row['consequents']
        checker = {}
        for consequent in consequents:
            if product_categories[consequent] not in checker:
                checker[product_categories[consequent]] = 1
            else:
                continue

            rec_object = {
                'product': consequent,
                'product_category' : product_categories[consequent],
                'confidence': row['confidence']
            }
            recommendation_dict[key].append(rec_object)


In [148]:
import pprint
pprint.pprint(recommendation_dict)

{'Almond Croissant': [{'confidence': 0.4840764331210191,
                       'product': 'Dark chocolate',
                       'product_category': 'Packaged Chocolate'},
                      {'confidence': 0.4585987261146497,
                       'product': 'Latte',
                       'product_category': 'Coffee'},
                      {'confidence': 0.4490445859872611,
                       'product': 'Cappuccino',
                       'product_category': 'Coffee'}],
 'Cappuccino': [{'confidence': 0.3008298755186722,
                 'product': 'Chocolate syrup',
                 'product_category': 'Flavours'},
                {'confidence': 0.2821576763485477,
                 'product': 'Carmel syrup',
                 'product_category': 'Flavours'},
                {'confidence': 0.2479253112033195,
                 'product': 'Hazelnut syrup',
                 'product_category': 'Flavours'},
                {'confidence': 0.1545643153526971,
                 'pr

In [149]:
import json 
with open('api/objects/recommendation_dict.json', 'w') as f:
    json.dump(recommendation_dict, f)